In [1]:
# importing the required packages and functions
import PyPDF2, numpy as np, pandas as pd # NOTE: Use the version 2.12.1 of PyPDF2 as the newer versions have deprecated many functionalities
from PyPDF2 import PdfFileReader
import re
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
pwd

'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023'

In [4]:
my_dir = 'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023\\'

In [5]:
my_dir = 'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023\\'
# Creating a "cleaning" function which we'll use to clean comments
def cleaner(x):
    x = str(x)
    x = re.sub(r'\[[0-9]*\]',' ',x)
    x = re.sub(r'\s+',' ',x)
    x = x.lower()
    x = re.sub(r'\d',' ',x)
    x = re.sub(r'\s+',' ',x)
    return x

In [6]:
# Creating a function to convert the contents of a pdf file into a list
def pdf_to_text(pdf):
    f = open(pdf,mode='rb')
    pdf_text = []

    pdf_reader3 = PyPDF2.PdfFileReader(f)

    for p in range(pdf_reader3.numPages):
        page = pdf_reader3.getPage(p)
        pdf_text.append(page.extractText())

    f.close()
    all_text = ' '.join(pdf_text).replace('\n','')
    cleaned_text = cleaner(all_text).split(' re: ')[-1]
    
    return cleaned_text

In [7]:
import os

In [8]:
os.listdir(my_dir)

['.ipynb_checkpoints',
 'api.py',
 'EPA-HQ-OW-2017-0300-0939_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1420_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1425_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1431_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1459_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1467_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1475_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1476_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1846_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1850_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1854_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1864_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1867_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1871_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1872_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1882_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1883_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1886_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1887_attachment_1.pdf',
 'Master-2.ipynb',
 'Master-3-Full-API.ipynb',
 'Master-3.ipynb',
 'Master-4-With HF APIs.ip

In [9]:
pdfs = []
for i in os.listdir(my_dir):
    if '.pdf' in i:
        pdfs.append(i)

In [10]:
pdfs

['EPA-HQ-OW-2017-0300-0939_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1420_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1425_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1431_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1459_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1467_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1475_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1476_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1846_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1850_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1854_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1864_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1867_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1871_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1872_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1882_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1883_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1886_attachment_1.pdf',
 'EPA-HQ-OW-2017-0300-1887_attachment_1.pdf']

In [11]:
text_of_pdfs = []
for i in pdfs:
    text_of_pdfs.append(pdf_to_text(i))

Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


In [12]:
len(text_of_pdfs)

19

In [13]:
len(pdfs)

19

In [14]:
#text_of_pdfs

In [15]:
df_comments = pd.DataFrame({'Comment':text_of_pdfs})

In [16]:
df_comments

,Comment
0,comments regarding the lead service line defin...
1,national primary drinking water regulations: p...
2,state of nevada department of conservation & ...
3,"february , mr. david ross assistant administr..."
4,national primary drinking water regulations: p...
5,national primary drinking water regulations: p...
6,proposed lead and copper rule revisions – epa ...
7,national primary drinking water regulations: p...
8,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...


## Sentence Similarity

In [17]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # using tokenizer from pretrained model
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # Assembling the model with pre-trained weights

In [18]:
# Creating an embedding function (basically vectorizing every word in every sentence in order to grab granular details present in each sentence)
def give_embed(sentences):
    def mean_pooling(model_output, attention_mask):

        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    #print("Sentence embeddings:")
    return sentence_embeddings


In [19]:
l1 = df_comments['Comment'].tolist() # maing a list of all the comments

In [20]:
l1 = df_comments['Comment'].tolist() # maing a list of all the comments
# Ensuring that all the comments belong to string datatype
l2 = []
for i in l1:
    l2.append(str(i))

In [21]:
series_embeds = give_embed(l2) # Applying our embedding function on our corpus

In [22]:
series_embeds = give_embed(l2) # Applying our embedding function on our corpus
# Evaluating the similarity for each combination of comments using the cosine similarity method 
sent1 = []
sent2 = []
similarity = []

for i in range(len(l2)-1):
    #sent1.append(l2[i])
    for j in range(i+1, len(l2)):
        sent1.append(l2[i])
        sent2.append(l2[j])
        similarity.append(cosine_similarity(series_embeds[i].reshape(1,-1) , series_embeds[j].reshape(1,-1) )[0][0])

In [23]:
len(sent1)

171

In [24]:
len(sent2)

171

In [25]:
len(similarity)

171

In [26]:
# creating the dataframe for all combination of comments along with their similarity scores
df_sentence_similarity = pd.DataFrame({'Docuemnt1':sent1, 'Document2': sent2,'Cosine_Similarity_Score': similarity}) 

In [27]:
df_sentence_similarity

,Docuemnt1,Document2,Cosine_Similarity_Score
0,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.737591
1,comments regarding the lead service line defin...,state of nevada department of conservation & ...,0.686464
2,comments regarding the lead service line defin...,"february , mr. david ross assistant administr...",0.586471
3,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.780700
4,comments regarding the lead service line defin...,national primary drinking water regulations: p...,0.736849
...,...,...,...
166,proposed delay of effective and compliance dat...,revisions to lead & copper na onal primary dri...,0.696102
167,proposed delay of effective and compliance dat...,docket id no. epa-hq-ow- - april jeff cohen s...,0.734126
168,proposed delay of effective and compliance dat...,revisions to lead & copper na onal primary dri...,0.584698
169,proposed delay of effective and compliance dat...,docket id no. epa-hq-ow- - april jeff cohen s...,0.596619


In [28]:
df_sentence_similarity[df_sentence_similarity['Cosine_Similarity_Score'] < 0.5]

,Docuemnt1,Document2,Cosine_Similarity_Score
40,state of nevada department of conservation & ...,epn comments on pr oposal to delay ef fecti v ...,0.440766
55,"february , mr. david ross assistant administr...",epn comments on pr oposal to delay ef fecti v ...,0.375158
57,"february , mr. david ross assistant administr...","epa, national primary drinking water regulati...",0.474698
61,"february , mr. david ross assistant administr...",comment to proposed rulemaking docket id no. e...,0.482838
63,"february , mr. david ross assistant administr...",proposed delay of effective and compliance dat...,0.488017
64,"february , mr. david ross assistant administr...",revisions to lead & copper na onal primary dri...,0.497157
94,proposed lead and copper rule revisions – epa ...,epn comments on pr oposal to delay ef fecti v ...,0.486376
105,national primary drinking water regulations: p...,epn comments on pr oposal to delay ef fecti v ...,0.485264
116,epn comments on pr oposal to delay ef fecti v ...,national primary drinking water regulations: p...,0.403843
118,epn comments on pr oposal to delay ef fecti v ...,national primary drinking water regulations: p...,0.486897


## KEYWORD EXTRACTION
ml6team/keyphrase-extraction-distilbert-inspec

In [29]:
from transformers import (
    TokenClassificationPipeline,
    AutoModelForTokenClassification,
    AutoTokenizer,
)
from transformers.pipelines import AggregationStrategy
#import numpy as np

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, model_outputs):
        results = super().postprocess(
            model_outputs=model_outputs,
            aggregation_strategy=AggregationStrategy.FIRST,
        )
        return np.unique([result.get("word").strip() for result in results])

In [30]:
# Importing the model and extractor from huggingface.co
model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
extractor = KeyphraseExtractionPipeline(model=model_name)

In [31]:
# Creating a keyword extracting function
def key_word_extractor(x):
    try:
        if len(x) > 0:
            return extractor(x)
        else:
            return np.nan
    except:
        return np.nan

In [32]:
df_comments['keywords'] = df_comments['Comment'].apply(lambda x: key_word_extractor(x)) # Extracting out the keywords from each comment

In [33]:
df_comments

,Comment,keywords
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu..."
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p..."
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme..."
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat..."
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co..."
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan..."
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day..."
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions..."
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur..."
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont..."


In [34]:
# Creating a list which contains the number of keywords of each comment
key_cnt = []
for i in range(len(df_comments)):
    try:
        key_cnt.append(len(df_comments['keywords'][i]))
    except:
        key_cnt.append(0)

In [35]:
#key_cnt

In [36]:
df_comments['key_count'] = key_cnt # Creating a new column which contains the number of keywords in each comment

In [37]:
df_comments

,Comment,keywords,key_count
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18


In [38]:
rev_len = [len(x.split()) if x is not np.nan else 0 for x in df_comments['Comment']]  # Creating a list which contains the number of words in each comment

In [39]:
#rev_len

In [40]:
df_comments['review_length'] = rev_len # Creating a new column which contains the number of words in each comment

In [41]:
df_comments

,Comment,keywords,key_count,review_length
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409


In [42]:
df_comments['proportion_keywords'] = 100 * df_comments['key_count']/df_comments['review_length'] # proportion of key words present in each comment depending on the context of each comment

In [43]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014


# SENTIMENT ANALYSIS

In [44]:
sia = SentimentIntensityAnalyzer() # Creating an instance of SentimentIntensityAnalyzer function to analyze the sentiment of each comment by calling its polarity_scores method

In [45]:

# Creating a function to bin a compound polarity score into 9 categories namely: Extremely Negative, Very Negative, Negative, Neutral-Negative, Neutral, Neutral-Positive, Positive, Very Positive, Extremely Positive
def final_sent_calc(x):
    
    def estimator(a):
        return float(str(a)[:6])
    def estimator2(b):
        return float(str(b)[:7])

    if x in np.asarray(pd.Series(np.arange(-1.001, -0.778,0.0001)).apply(lambda x: estimator2(x))):
        return 'Extremely Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.778, -0.556,0.0001)).apply(lambda x: estimator2(x))):
        return 'Very Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.556, -0.333,0.0001)).apply(lambda x: estimator2(x))):
        return 'Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.333, -0.111,0.0001)).apply(lambda x: estimator2(x))):
        return 'Neutral-Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.111, 0.000,0.0001)).apply(lambda x: estimator2(x))):
        return 'Neutral'
    elif x in np.asarray(pd.Series(np.arange(0, 0.111,0.0001)).apply(lambda x: estimator(x))):
        return 'Neutral'
    elif x in np.asarray(pd.Series(np.arange(0.111, 0.333,0.0001)).apply(lambda x: estimator(x))):
        return 'Neutral-Positive'
    elif x in np.asarray(pd.Series(np.arange(0.333, 0.556,0.0001)).apply(lambda x: estimator(x))):
        return 'Positive'
    elif x in np.asarray(pd.Series(np.arange(0.556, 0.778,0.0001)).apply(lambda x: estimator(x))):
        return 'Very Positive'
    elif x in np.asarray(pd.Series(np.arange(0.778, 1.001,0.0001)).apply(lambda x: estimator(x))):
        return 'Extremely Positive'

In [46]:
df_comments['sentiment_score'] = df_comments['Comment'].apply(lambda x: sia.polarity_scores(x)['compound']) # Applying polarity score method to get the compound polarity score of each comment

In [47]:
df_comments['sentiment'] = df_comments['sentiment_score'].apply(lambda x: final_sent_calc(x)) # Evaluating the sentiment type by applying the binning function

In [48]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive


In [49]:
#df_comments.to_csv('Master-General.csv')

In [50]:
#df_sentence_similarity.to_csv('Master-Similarity.csv')

## Summarization

In [51]:
#from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

In [52]:
def summary_func(x):
    word_count = int(len(x.split()))
    limit = 500
    if word_count > limit:
        return summarizer(' '.join(x.split()[:limit]))[0]['summary_text']
    else:
        return summarizer(x)[0]['summary_text']

In [53]:
df_comments['summary'] = df_comments['Comment'].apply(lambda x: summary_func(x))

In [54]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."


In [ ]:
df

In [55]:
df_comments['summary'][0]

'Environmental defense fund submits comments on lead service line definition and inventory aspects of the environmental protection agency’s (epa) proposed revisions to its lead and copper rule (lcr) The agency published proposed revisions in the november , , federa l register at fed reg. , and is accepting comments at docket no. epa -hq-ow- - . edf’ s mission is to preserve the natural systems on which all life depends.'

In [56]:
df_comments['keywords'][0]

array(['- lead', 'comments', 'connectors', 'copper rule',
       'customers misleading', 'economics', 'environmental defense fund',
       'environmental problems', 'environmental protection agency',
       'goosenecks', 'health', 'health program', 'human health',
       'lead service line', 'lead service line definition',
       'national primary drinking water regulations', 'pigtails',
       'policy experts', 'pollution', 'toxic chemicals', 'water flow',
       'water systems'], dtype='<U43')

In [57]:
#df_comments['Comment'][0]

In [58]:
#pd.DataFrame(df_comments.iloc[4:7,:].to_dict())

In [59]:
#df_comments

In [60]:
def search_keyword(search_term='revisions'):
    search = search_term
    search = search.lower()
    #idxs = []
    df = pd.DataFrame()
   # cnt=0
    for i in range(len(df_comments['keywords'])):
        if search in df_comments['keywords'][i]:
            #cnt += 1
            df = df.append(df_comments.iloc[i,:])
        else:
            continue
   
    return df

In [61]:
search_keyword('copper rule revisions')

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...
13,comments on proposed delay of effective and co...,"[compliance, copper rule, copper rule revision...",11,3424,0.321262,0.9986,Extremely Positive,The natural resources defense council (nrdc) s...
15,proposed delay of effective and compliance dat...,[california state water resources control boar...,12,945,1.269841,0.9967,Extremely Positive,California state water resources control board...


In [62]:
search_keyword('compliance')

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...
13,comments on proposed delay of effective and co...,"[compliance, copper rule, copper rule revision...",11,3424,0.321262,0.9986,Extremely Positive,The natural resources defense council (nrdc) s...
15,proposed delay of effective and compliance dat...,[california state water resources control boar...,12,945,1.269841,0.9967,Extremely Positive,California state water resources control board...
16,proposed delay of effective and compliance dat...,"[alaska drinking water program, alaska drinkin...",13,231,5.627706,0.9847,Extremely Positive,The state and territorial primacy agencies are...
18,docket id no. epa-hq-ow- - april jeff cohen s...,"[brass faucets, compliance, contaminant, corro...",14,770,1.818182,0.2006,Neutral-Positive,i worked for the us epa in various programs an...


# use gensim word2vec to search in a better using 'most_similar_word'

In [63]:
all_keywords = []
for i in range(len(df_comments['keywords'])):
    for j in df_comments['keywords'][i]:
        all_keywords.append(j)

In [64]:
all_keywords_str = ' '.join(all_keywords)

In [65]:
import nltk
nltk.download('punkt')
from gensim.models import Word2Vec
from nltk.corpus import stopwords
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rsinha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [66]:
def my_tokenizer(x):
    x = nltk.sent_tokenize(x)

    sentences = [nltk.word_tokenize(sentence) for sentence in x]

    for i in range(len(sentences)):
        sentences[i] = [word for word in sentences[i] if word not in stopwords.words('english')]
    return sentences

In [67]:
keywords_tokenized = my_tokenizer(all_keywords_str)

In [68]:
#keywords_tokenized

In [69]:
tokenized_str = []
for i in range(len(keywords_tokenized)):
    tokenized_str.append(' '.join(keywords_tokenized[i]))
#tokenized_str

In [70]:
tokenized_str_final = ' '.join(tokenized_str)
#tokenized_str_final

In [71]:
search_model = Word2Vec(keywords_tokenized, min_count=1)

In [72]:
'changes' in tokenized_str_final

False

In [73]:
similar = search_model.wv.most_similar('copper')
similar

[('sewer', 0.26975303888320923),
 ('agency', 0.2586798369884491),
 ('phd', 0.2455655187368393),
 ('laws', 0.22235117852687836),
 ('household', 0.22090166807174683),
 ('conformity', 0.22009442746639252),
 ('tracking', 0.2133825272321701),
 ('docket', 0.2117016315460205),
 ('protective', 0.2098371684551239),
 ('act', 0.20787814259529114)]

In [74]:
search_model.wv.most_similar_cosmul(['revisions','delay'])

[('techniques', 0.3857749104499817),
 ('contamination', 0.3642527759075165),
 ('education', 0.3492853343486786),
 ('voluntary', 0.34888383746147156),
 ('county', 0.34589874744415283),
 ('pigtails', 0.34202420711517334),
 ('rity', 0.3409615755081177),
 ('believ', 0.33576521277427673),
 ('housing', 0.3342360258102417),
 ('laws', 0.33365651965141296)]

In [75]:
def search_finder_advanced(search_term="revisions"):
    search_term = search_term.lower()
    df = pd.DataFrame()
    try:
        similar_terms = []
        for term in search_term.split():
            similar_terms.append(term)
        #similar_terms.append(search_term)
        #for n in similar_terms:

        for a,b in search_model.wv.most_similar(similar_terms):
            similar_terms.append(a.lower())
        for i in range(len(df_comments['keywords'])):
            results = [j for j in similar_terms if j in ' '.join(df_comments['keywords'][i])]
            if len(results) > 0:
                df = df.append(df_comments.iloc[i,:])
        if len(df) == 0:
            print('Test-1')
            for i in search_term.split():
                similar_terms.append(i)
            for i in range(len(df_comments['Comment'])):
                results = [j for j in similar_terms if j in df_comments['Comment'][i]]
                df = df.append(df_comments.iloc[i,:])
        print('Test-2')
        return df
    except:
        print('Test-3')
        search = search_term
        search = search.lower()
        #idxs = []
        df = pd.DataFrame()
       # cnt=0
        for i in range(len(df_comments['Comment'])):
            if search in df_comments['Comment'][i]:
                #cnt += 1
                df = df.append(df_comments.iloc[i,:])
            else:
                continue
        return df

In [76]:
search_finder_advanced('revisions')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [77]:
search_finder_advanced('copper rule revisions')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [78]:
search_finder_advanced('copper')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [79]:
search_finder_advanced('compliance')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...


In [80]:
search_finder_advanced('alaska')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...
14,comment to proposed rulemaking docket id no. e...,"[children, community empowerment, dutch town s...",18,1681,1.070791,0.9833,Extremely Positive,Comments support the proposed delay of the lcr...
16,proposed delay of effective and compliance dat...,"[alaska drinking water program, alaska drinkin...",13,231,5.627706,0.9847,Extremely Positive,The state and territorial primacy agencies are...
17,revisions to lead & copper na onal primary dri...,"[coalibon, corrosion control treatment irrespe...",13,25933,0.050129,0.9960,Extremely Positive,The EPA’s proposed revisions to the lead and c...
18,docket id no. epa-hq-ow- - april jeff cohen s...,"[brass faucets, compliance, contaminant, corro...",14,770,1.818182,0.2006,Neutral-Positive,i worked for the us epa in various programs an...


In [81]:
search_keyword('alaska')

""


In [82]:
search_finder_advanced('changes')

Test-3


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...
12,"lcr epa revision of hq-ow - comments april , ...","[compliance, corrosion control, cotruvo phd, l...",11,1346,0.817236,0.9826,Extremely Positive,Lcr epa revision of hq-ow was poorly conc eive...
13,comments on proposed delay of effective and co...,"[compliance, copper rule, copper rule revision...",11,3424,0.321262,0.9986,Extremely Positive,The natural resources defense council (nrdc) s...
17,revisions to lead & copper na onal primary dri...,"[coalibon, corrosion control treatment irrespe...",13,25933,0.050129,0.9960,Extremely Positive,The EPA’s proposed revisions to the lead and c...


In [83]:
search_finder_advanced('delay')

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."
10,"epa, national primary drinking water regulati...","[compliance, copper rule revisions, costs, fcc...",7,979,0.715015,-0.9857,Extremely Negative,The agency’s proposal is procedurally and subs...


In [84]:
search_finder_advanced("dutch")

Test-2


,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
11,national primary drinking water regulations: p...,"[bermuda, children, compliance, control, coppe...",16,2024,0.790514,0.9873,Extremely Positive,The international society for environmental ep...


In [85]:
df_comments

,Comment,keywords,key_count,review_length,proportion_keywords,sentiment_score,sentiment,summary
0,comments regarding the lead service line defin...,"[- lead, comments, connectors, copper rule, cu...",22,6313,0.348487,0.9998,Extremely Positive,Environmental defense fund submits comments on...
1,national primary drinking water regulations: p...,"[beach, childcare facilities, city, city o f p...",26,600,4.333333,0.9938,Extremely Positive,The city of pompano beach appreciates the oppo...
2,state of nevada department of conservation & ...,"[carson city, copper rule revisions, environme...",11,506,2.173913,0.9945,Extremely Positive,The state of nevada offers the attached remark...
3,"february , mr. david ross assistant administr...","[administrative, advancement act, american nat...",42,888,4.729730,0.9982,Extremely Positive,iapmo is a -year-old trade association that fo...
4,national primary drinking water regulations: p...,"[copper rule, copper rule revisions docket, co...",30,2256,1.329787,0.9876,Extremely Positive,Public comments attached in response to the us...
5,national primary drinking water regulations: p...,"[columbia, copper rule revisions, full, galvan...",13,3092,0.420440,0.9970,Extremely Positive,The district of columbia water and sewer autho...
6,proposed lead and copper rule revisions – epa ...,"[copper rule revisions, corrosion control, day...",18,4679,0.384698,0.9996,Extremely Positive,Missouri department of natural resources provi...
7,national primary drinking water regulations: p...,"[community, copper rule, copper rule revisions...",13,1020,1.274510,0.9779,Extremely Positive,The springfield water and sewer commission (co...
8,epn comments on pr oposal to delay ef fecti v ...,"[ann, believ, compliance, compliancedates, cur...",19,645,2.945736,0.9727,Extremely Positive,epn comments on pr oposal to delay ef fecti v ...
9,national primary drinking water regulations: p...,"[children, copper rule revisions, disease cont...",18,3409,0.528014,0.9991,Extremely Positive,"Mayors, counties, cities and the mayors of the..."


In [86]:
pwd

'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023'

In [87]:
#df_comments.sort_values('key_count',ascending=False)

In [88]:
#df_comments.to_csv('Master-General-3.csv',index=False)

In [89]:
import jsonify

In [90]:
#df_comments.to_dict()

# API

In [1]:
# importing the required packages and functions
import PyPDF2, numpy as np, pandas as pd # NOTE: Use the version 2.12.1 of PyPDF2 as the newer versions have deprecated many functionalities
from PyPDF2 import PdfFileReader
import re
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModel, pipeline , TokenClassificationPipeline , AutoModelForTokenClassification
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")
import os
from transformers.pipelines import AggregationStrategy
from flask import Flask, request, render_template, jsonify

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rsinha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\rsinha\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
my_dir = 'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023\\'
# Creating a "cleaning" function which we'll use to clean comments
def cleaner(x):
    x = str(x)
    x = re.sub(r'\[[0-9]*\]',' ',x)
    x = re.sub(r'\s+',' ',x)
    x = x.lower()
    x = re.sub(r'\d',' ',x)
    x = re.sub(r'\s+',' ',x)
    return x

# Creating a function to convert the contents of a pdf file into a list
def pdf_to_text(pdf):
    f = open(pdf,mode='rb')
    pdf_text = []

    pdf_reader3 = PyPDF2.PdfFileReader(f)

    for p in range(pdf_reader3.numPages):
        page = pdf_reader3.getPage(p)
        pdf_text.append(page.extractText())

    f.close()
    all_text = ' '.join(pdf_text).replace('\n','')
    cleaned_text = cleaner(all_text).split(' re: ')[-1]
    
    return cleaned_text

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # using tokenizer from pretrained model
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # Assembling the model with pre-trained weights

# Creating an embedding function (basically vectorizing every word in every sentence in order to grab granular details present in each sentence)
def give_embed(sentences):
    def mean_pooling(model_output, attention_mask):

        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    
    #tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # using tokenizer from pretrained model
    #model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # Assembling the model with pre-trained weights
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling
    sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    #print("Sentence embeddings:")
    return sentence_embeddings

# Define keyphrase extraction pipeline
class KeyphraseExtractionPipeline(TokenClassificationPipeline):
    def __init__(self, model, *args, **kwargs):
        super().__init__(
            model=AutoModelForTokenClassification.from_pretrained(model),
            tokenizer=AutoTokenizer.from_pretrained(model),
            *args,
            **kwargs
        )

    def postprocess(self, model_outputs):
        results = super().postprocess(
            model_outputs=model_outputs,
            aggregation_strategy=AggregationStrategy.FIRST,
        )
        return np.unique([result.get("word").strip() for result in results])
    
# Creating a keyword extracting function
def key_word_extractor(x):
    try:
        if len(x) > 0:
            return extractor(x)
        else:
            return np.nan
    except:
        return np.nan
    
# Creating a function to bin a compound polarity score into 9 categories namely: Extremely Negative, Very Negative, Negative, Neutral-Negative, Neutral, Neutral-Positive, Positive, Very Positive, Extremely Positive
def final_sent_calc(x):
    
    def estimator(a):
        return float(str(a)[:6])
    def estimator2(b):
        return float(str(b)[:7])

    if x in np.asarray(pd.Series(np.arange(-1.001, -0.778,0.0001)).apply(lambda x: estimator2(x))):
        return 'Extremely Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.778, -0.556,0.0001)).apply(lambda x: estimator2(x))):
        return 'Very Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.556, -0.333,0.0001)).apply(lambda x: estimator2(x))):
        return 'Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.333, -0.111,0.0001)).apply(lambda x: estimator2(x))):
        return 'Neutral-Negative'
    elif x in np.asarray(pd.Series(np.arange(-0.111, 0.000,0.0001)).apply(lambda x: estimator2(x))):
        return 'Neutral'
    elif x in np.asarray(pd.Series(np.arange(0, 0.111,0.0001)).apply(lambda x: estimator(x))):
        return 'Neutral'
    elif x in np.asarray(pd.Series(np.arange(0.111, 0.333,0.0001)).apply(lambda x: estimator(x))):
        return 'Neutral-Positive'
    elif x in np.asarray(pd.Series(np.arange(0.333, 0.556,0.0001)).apply(lambda x: estimator(x))):
        return 'Positive'
    elif x in np.asarray(pd.Series(np.arange(0.556, 0.778,0.0001)).apply(lambda x: estimator(x))):
        return 'Very Positive'
    elif x in np.asarray(pd.Series(np.arange(0.778, 1.001,0.0001)).apply(lambda x: estimator(x))):
        return 'Extremely Positive'
    
def summary_func(x):
    word_count = int(len(x.split()))
    limit = 500
    if word_count > limit:
        return summarizer(' '.join(x.split()[:limit]))[0]['summary_text']
    else:
        return summarizer(x)[0]['summary_text']
    

In [3]:
app = Flask(__name__)

@app.route('/cpt')
def comment_process_tool():
    my_dir = 'C:\\Users\\rsinha\\Desktop\\Comment Tool\\OneDrive_1_2-10-2023\\'
    pdfs = []
    for i in os.listdir(my_dir):
        if '.pdf' in i:
            pdfs.append(i)
    
    text_of_pdfs = []
    for i in pdfs:
        text_of_pdfs.append(pdf_to_text(i))

    df_comments = pd.DataFrame({'Comment':text_of_pdfs}) 
    
    #tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # using tokenizer from pretrained model
   #model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2') # Assembling the model with pre-trained weights
    
    l1 = df_comments['Comment'].tolist() # maing a list of all the comments
    # Ensuring that all the comments belong to string datatype
    l2 = []
    for i in l1:
        l2.append(str(i))
        
    series_embeds = give_embed(l2) # Applying our embedding function on our corpus
    # Evaluating the similarity for each combination of comments using the cosine similarity method 
    sent1 = []
    sent2 = []
    similarity = []

    for i in range(len(l2)-1):
        #sent1.append(l2[i])
        for j in range(i+1, len(l2)):
            sent1.append(l2[i])
            sent2.append(l2[j])
            similarity.append(cosine_similarity(series_embeds[i].reshape(1,-1) , series_embeds[j].reshape(1,-1) )[0][0])
            
    # creating the dataframe for all combination of comments along with their similarity scores
    df_sentence_similarity = pd.DataFrame({'Docuemnt1':sent1, 'Document2': sent2,'Cosine_Similarity_Score': similarity}) 
    
    # Importing the model and extractor from huggingface.co
    model_name = "ml6team/keyphrase-extraction-distilbert-inspec"
    extractor = KeyphraseExtractionPipeline(model=model_name)
    
    df_comments['keywords'] = df_comments['Comment'].apply(lambda x: key_word_extractor(x)) # Extracting out the keywords from each comment
    
    # Creating a list which contains the number of keywords of each comment
    key_cnt = []
    for i in range(len(df_comments)):
        try:
            key_cnt.append(len(df_comments['keywords'][i]))
        except:
            key_cnt.append(0)
            
    df_comments['key_count'] = key_cnt # Creating a new column which contains the number of keywords in each comment
    
    rev_len = [len(x.split()) if x is not np.nan else 0 for x in df_comments['Comment']]  # Creating a list which contains the number of words in each comment
    df_comments['review_length'] = rev_len # Creating a new column which contains the number of words in each comment
    
    df_comments['proportion_keywords'] = 100 * df_comments['key_count']/df_comments['review_length'] # proportion of key words present in each comment depending on the context of each comment
    
    sia = SentimentIntensityAnalyzer() # Creating an instance of SentimentIntensityAnalyzer function to analyze the sentiment of each comment by calling its polarity_scores method
    
    df_comments['sentiment_score'] = df_comments['Comment'].apply(lambda x: sia.polarity_scores(x)['compound']) # Applying polarity score method to get the compound polarity score of each comment
    df_comments['sentiment'] = df_comments['sentiment_score'].apply(lambda x: final_sent_calc(x)) # Evaluating the sentiment type by applying the binning function
    
    #from transformers import pipeline
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    
    df_comments['summary'] = df_comments['Comment'].apply(lambda x: summary_func(x))
    
    return jsonify(df_comments.to_json())



if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [8]:
import sentence_transformers

In [9]:
sentence_transformers.__version__

'2.2.2'